In [ ]:
#IMPORT PACKAGES
import os.path
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import matplotlib.pyplot as plt

# from tabulate import tabulate

import json  
import copy

from sklearn.cluster import DBSCAN
from PIL import Image

from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context
import utils as utl
import utils.t8_utils as t8


# Grounding DINO
from groundingdino.util.inference import  annotate, load_image, predict
from torch.cuda.amp import autocast

# segment anything
# from segment_anything import build_sam, SamPredictor 


# diffusers
import torch


In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

## INPUTS

In [ ]:
#paths
path=Path(os.getcwd()).parents[0]

print(path)
input_folder_t4=path/'data'/'t4'/'train' 
input_folder_t6=path/'data'/'t6'/'train'
class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t8'/ 'train'
os.makedirs(output_folder, exist_ok=True)

ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"

#parameters
image_resolution = 0.01
class_id = 5
id_count = 4000

t_score = 0.55

TEXT_PROMPT = "Door"
BOX_TRESHOLD = 0.20
TEXT_TRESHOLD = 0.5

Import Classes

In [ ]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

Load Grounding Dino Model

In [ ]:
with autocast():
        groundingdino_model = t8.load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device="cuda:1")

## Processing

In [ ]:
point_cloud_files=utl.get_list_of_files(input_folder_t4,'.laz')
wall_files=utl.get_list_of_files(input_folder_t6,'.ttl')

In [8]:
for f_pcd in point_cloud_files:
    project = f_pcd.split("/")[-1].split(".")[-2].split("_small1")[0]
    for f_rdf in wall_files:
        if project == f_rdf.split("/")[-1].split(".")[-2].split("_walls")[0]:
            break
    print(f_pcd)
    print(f_rdf)
    doorNodes = []
    door_count = 0
    print(f'processing {ut.get_filename(f_pcd)}...') 
    wallNodes=tl.graph_path_to_nodes(f_rdf)
    for n in wallNodes:
        n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(n.orientedBounds)))
        n.pcd = None
    print(f'{len(wallNodes)} wallNodes detected!')
    
    wallNodes = t8.match_graph_with_las(f_pcd,class_dict, nodes = wallNodes, getResources=True, getNormals=False)
    
    pcd_count = 0
    for n in wallNodes:
        if not n.pcd == None:
            pcd_count += 1
    print(f'Matched {pcd_count} pointclouds to nodes')
    
    pointcloud = gmu.join_geometries([n.pcd for n in wallNodes])
    # print(pointcloud)
    # o3d.io.write_point_cloud(os.path.join(output_folder, "pointcloud.pcd"), pointcloud)
    # path = os.path.join(output_folder, "pointcloud.pcd")
    # print(f"Saved Pointcloud to {path}")
    for n in wallNodes: 
        n.derivedFrom = next((p for p in wallNodes if n.object_id == p.object_id), None)
        n.startpoint = ut.literal_to_array(n.start_pt) #np.asarray(n.start_pt[1:-1].split(), dtype=float)
        n.endpoint = ut.literal_to_array(n.end_pt) #np.asarray(n.end_pt[1:-1].split(), dtype=float)
        n.normal = ut.literal_to_array(n.normal) #np.asarray(n.normal[1:-1].split(), dtype=float)
        n.height = float(n.height)
        n.name = n.subject.split('///')[-1]
        if n.width == 0.127 or n.width == 0.2:
            n.singleFaced = True
        else:
            n.singleFaced = False
    
    #Create a messh from this point cloud 
    octree=pt.pcd_to_octree(gmu.join_geometries([n.pcd for n in wallNodes]),12) #if octree is None else octree

    # Calculate the size of each octree node based on octree depth and overall size
    def calculate_node_size(octree_depth, octree_size):
        num_voxels_per_dim = 2 ** octree_depth
        voxel_size = octree_size / num_voxels_per_dim
        return voxel_size

    voxel_size = calculate_node_size(octree.max_depth, octree.size)
    # print("Voxelsize = ", voxel_size)
    
    voxelmesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh
    # o3d.io.write_triangle_mesh(os.path.join(output_folder, "mesh.obj"), voxelmesh)
    # path = os.path.join(output_folder, "mesh.obj")
    # print(f"Saved Mesh to {path}")
    
    #Create a identity array containing the color so this can be retrieved afterwards
    original_colors=np.asarray(voxelmesh.vertex_colors)

    indices=np.asarray(voxelmesh.triangles)[:,0]
    triangle_colors=original_colors[indices]

    #append black color at the end of the array for the invalid hits
    triangle_colors=np.vstack((triangle_colors,np.array([0,0,0])))
    
    cpu_mesh=o3d.t.geometry.TriangleMesh.from_legacy(voxelmesh)
    # o3d.io.write_triangle_mesh(os.path.join(output_folder, "mesh.obj"), cpu_mesh)
    # path = os.path.join(output_folder, "mesh.obj")
    # print(f"Saved Mesh to {path}")
    
    # Create raycasting scene
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(cpu_mesh) 
    
    wall = 0
    for n in wallNodes:
        length = np.sqrt(np.sum((n.endpoint - n.startpoint)**2))
        surface = length * n.height
        image_size = (int(length / image_resolution), int(n.height / image_resolution))
        n.orthos = []
        
        if not surface < 3 and n.height > 1.5 and length > 0.8:  
            #Create an ortho of the dominant side of the wall      
            ortho = t8.create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = n.normal, scene=scene, triangle_colors = triangle_colors, show = False)
            ortho = t8.fill_black_pixels(ortho, region = 12)
            n.orthos.append(ortho)
            
            # if np.max(ortho) <= 1.0:
            #     ortho = ortho * 255.0
            # ortho_uint8 = ortho.astype(np.uint8)
            # Image.fromarray(ortho_uint8).save(os.path.join(output_folder,("Wall_" + str(n.object_id) + "-Face_" + str(0) +'.png')))
            # plt.imshow(n.orthos[0])
            
            #Also create an ortho of the other side of the wall
            if not n.singleFaced: #Single faced wall only needs one side
                ortho = t8.create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = -n.normal, scene=scene, triangle_colors = triangle_colors, dominant = False)
                ortho = t8.fill_black_pixels(ortho, region = 12)
                n.orthos.append(ortho)
                # if np.max(ortho) <= 1.0:
                #     ortho = ortho * 255.0
                # ortho_uint8 = ortho.astype(np.uint8)
                # Image.fromarray(ortho_uint8).save(os.path.join(output_folder,("Wall_" + str(n.object_id) + "-Face_" + str(1) +'.png')))
        # plt.show()
        wall +=1
    for n in wallNodes:
        n.boxes = []
        n.logits = []
        n.phrases = []
            
        if len(n.orthos) > 0:
            for ortho in n.orthos:
                
                boxes = None
                image = load_image(Image.fromarray((ortho * 255).astype(np.uint8)))

                boxes, logits, phrases = predict(
                    model=groundingdino_model, 
                    image=image, 
                    caption=TEXT_PROMPT, 
                    box_threshold=BOX_TRESHOLD, 
                    text_threshold=TEXT_TRESHOLD
                )
                n.boxes.append(boxes)
                n.logits.append(logits)
                n.phrases.append(phrases)
    wall = 0
    door_count = 0
    doorNodes = []
    

    for n in wallNodes:
        potential_door_boxes_wall = []
        potential_door_info_wall = []
        # print(f'Processing {n.name}')
        
        if (len(n.orthos) == 1 and len(n.boxes[0]) > 0) or len(n.orthos) == 2 and (len(n.boxes[0]) > 0 or len(n.boxes[1]) > 0) :
            for j, boxes in enumerate(n.boxes):
                potential_door_boxes_face = []
                potential_door_info_face = []
                if len(boxes) > 0: 
                    for i, box in enumerate(boxes):
                        probability = float(n.logits[j][i])
                        opening_width = round(int(np.asarray(box)[2]*n.orthos[j].shape[1])* image_resolution, 2)    
                        opening_height = round(int(np.asarray(box)[3]*n.orthos[j].shape[0]) * image_resolution, 2)
                        
                        detection_center_u = int(np.asarray(box)[0]*n.orthos[j].shape[1]) * image_resolution
                        detection_center_v = int(np.asarray(box)[1]*n.orthos[j].shape[0]) * image_resolution
                        
                        reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)
                        # print((str(j) + "-" +str(image)))
                        score = t8.is_door(probability, opening_width, opening_height, reference_level, prob_weight=0.15, width_weight=0.15, height_weight=0.3, ref_level_weight=0.3, print_overview = False)
                        if score >= 0:
                            # annotated_frame = annotate(image_source= n.orthos[j], boxes=box.unsqueeze(0), logits=torch.from_numpy(np.array([score])), phrases=[n.phrases[j][i]])
                            # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                            # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Wall_" + str(n.object_id) + "-Face_" + str(j) +"-"+ str(score) +'-DOOR.png')))
                            # image += 1
                            
                            box1 = copy.deepcopy(box)
                            box = box.unsqueeze(0)

                            if j == 1:
                                box1[0] = 1-box1[0]

                            potential_door_boxes_face.append(np.asarray(box1))
                            potential_door_info_face.append([opening_width, opening_height, image, reference_level, score])
                                
                potential_door_boxes_wall.append(potential_door_boxes_face)
                potential_door_info_wall.append(potential_door_info_face)
            #merge largely overlapping bounding boxes
            
            for i, face_boxes in enumerate(potential_door_boxes_wall):
                if len(face_boxes) > 2:
                    face_boxes, potential_door_info_wall[i] = t8.find_and_merge_high_iou_boxes(face_boxes, threshold=0.8, info = potential_door_info_wall[i])
        
            #For Double faced walls look if the detection is found on both sides.
            if len(potential_door_boxes_wall) == 2 and not len(potential_door_boxes_wall[0]) == 0 and not len(potential_door_boxes_wall[1]) == 0:
                # print("Double Faced")
                matches, unmatched_boxes0, unmatched_boxes1 = t8.find_best_matches(potential_door_boxes_wall[0], potential_door_boxes_wall[1], potential_door_info_wall[0], potential_door_info_wall[1], iou_weight=0.33, param_weight=0.33, doorness_weight=0.33)

                for id0, id1, bestscore in matches:
                    if not id0 == None and not id1 == None:
                        # compare the parameters of the different matches
                        info0 = potential_door_info_wall[0][id0]
                        info1 = potential_door_info_wall[1][id1]
            
                        # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(np.asarray([potential_door_boxes_wall[0][id0]])), logits=torch.from_numpy(np.array([info0[-1]])), phrases=[n.phrases[0][id0]])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Wall_" + str(wall) + "-Face_0-" +str(image) +"-"+  str(info0[-1]) +'-DOOR.png')))
                        # image += 1
                        
                        # annotated_frame = annotate(image_source= n.orthos[1], boxes=torch.from_numpy(np.asarray([potential_door_boxes_wall[1][id1]])), logits=torch.from_numpy(np.array([info1[-1]])), phrases=[n.phrases[1][id1]])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Wall_" + str(wall) + "-Face_1-" +str(image) +"-"+  str(info1[-1]) +'-DOOR.png')))
                        # image += 1
                        
                        detectionbox = t8.combine_boxes(potential_door_boxes_wall[0][id0], potential_door_boxes_wall[1][id1])
                        probability = (potential_door_info_wall[0][id0][4]+ potential_door_info_wall[1][id1][4])/2
                        
                        opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[0], image_resolution = image_resolution)
                        # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %((str(count)), bestscore, reference_level, opening_width, opening_height))
                        score = t8.is_door(probability, opening_width, opening_height, reference_level, prob_weight=0.15, width_weight=0.15, height_weight=0.3, ref_level_weight=0.3)
                        if score > t_score:
                            boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                            
                            doornode = BIMNode(
                                name= "Doors_" + str(door_count),
                                axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                                startpoint= boundaryPoints[0],
                                endpoint= boundaryPoints[1],
                                doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                                height = opening_height,
                                doornessScore = score,
                                singleFaced = False,
                                depth = n.width,
                                object_id = class_id *4000 + door_count,
                                blackness = percentage_black_pixles,
                                reference_level = reference_level,
                                host = n)
                            
                            doorNodes.append(doornode)
                            
                            # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                            # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                            # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                            door_count += 1 
                        
                #If the detection is not found on both sides of the wall but it is a double faced wall we add a penalty
                for id0 in unmatched_boxes0: #Two faced walls with a detection on only one side
                    score = potential_door_info_wall[0][id0][4]-0.2
                    if score > t_score:
                        detectionbox = np.array([potential_door_boxes_wall[0][id0]])
                        
                        opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[0], image_resolution = image_resolution)
                        
                        boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
            
                        doornode = BIMNode(
                            name= "Doors_" + str(door_count),
                            axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                            startpoint= boundaryPoints[0],
                            endpoint= boundaryPoints[1],
                            doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                            height = opening_height,
                            doornessScore = score,
                            singleFaced = True,
                            depth = n.width,
                            object_id = class_id *4000 + door_count,
                            blackness = percentage_black_pixles,
                            reference_level = reference_level,
                            host = n)
                        
                        doorNodes.append(doornode)
                        # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, reference_level, opening_width, opening_height))
                        # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                        door_count += 1 
                    
                for id1 in unmatched_boxes1: #Two faced walls with a detection on only one side
                    score = potential_door_info_wall[1][id1][4]-0.2
                    if score > t_score:
                        detectionbox = np.array([potential_door_boxes_wall[1][id1]])

                        opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[1], image_resolution = image_resolution)
                        
                        boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                            
                        doornode = BIMNode(
                            name= "Doors_" + str(door_count),
                            axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                            startpoint= boundaryPoints[0],
                            endpoint= boundaryPoints[1],
                            doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                            height = opening_height,
                            doornessScore = score,
                            singleFaced = True,
                            depth = n.width,
                            object_id = class_id *4000 + door_count,
                            blackness = percentage_black_pixles,
                            reference_level = reference_level,
                            host = n)
                        
                        doorNodes.append(doornode)
                        # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, reference_level, opening_width, opening_height))
                        # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                        door_count += 1 

            elif len(potential_door_boxes_wall) > 0: #For single faced walls
                if len(potential_door_boxes_wall) == 1:
                    for i, box in enumerate(potential_door_boxes_wall[0]):
                        #This are single faced walls so the penalty is less.
                        score = potential_door_info_wall[0][i][4]-0.1
                        # image_resource = t8.extract_box_with_margin(n.orthos[0], np.array([box])[0])
                        # image_resource = image_resource[...,::-1] # BGR to RGB
                        # Image.fromarray(image_resource).save(os.path.join(output_folder,("BlackDoor_" + str(door_count) + '.png')))
                        # bl_px = t8.calculate_percentage_black_pixels(image_resource)
                        
                        if score > t_score: #and bl_px > 0.6:
                            detectionbox = np.array([box])
                            
                            opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[0], image_resolution = image_resolution)
                            
                            boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                                
                            doornode = BIMNode(
                                name= "Doors_" + str(door_count),
                                axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                                startpoint= boundaryPoints[0],
                                endpoint= boundaryPoints[1],
                                doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                                height = opening_height,
                                doornessScore = score,
                                singleFaced = True,
                                depth = n.width,
                                object_id = class_id *4000 + door_count,
                                blackness = percentage_black_pixles,
                                reference_level = reference_level,
                                host = n)
                            
                            doorNodes.append(doornode)
                            # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, reference_level, opening_width, opening_height))
                            # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                            # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                            # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                            door_count += 1 
            
                if len(potential_door_boxes_wall) == 2:    
                        for i, box in enumerate(potential_door_boxes_wall[1]):
                            #This are single faced walls so the penalty is less.
                            score = potential_door_info_wall[1][i][4]-0.1
                            
                            if score > t_score:
                                detectionbox = np.array([box])
                                
                                opening_width, opening_height, detection_center_u, detection_center_v, reference_level, percentage_black_pixles = t8.compute_door_parameters(detectionbox, ortho = n.orthos[1], image_resolution = image_resolution)
                                    
                                boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                                
                                
                                doornode = BIMNode(
                                    name= "Doors_" + str(door_count),
                                    axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1]),
                                    startpoint= boundaryPoints[0],
                                    endpoint= boundaryPoints[1],
                                    doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2),
                                    height = opening_height,
                                    doornessScore = score,
                                    singleFaced = True,
                                    depth = n.width,
                                    object_id = class_id *4000 + door_count,
                                    blackness = percentage_black_pixles,
                                    reference_level = reference_level,
                                    host = n)
                                
                                doorNodes.append(doornode)
                                # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, reference_level, opening_width, opening_height))
                                # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([doornode.doornessScore])), phrases=["Door"])
                                # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                                # Image.fromarray(annotated_frame).save(os.path.join(output_folder,("Door_" + str(door_count) + '.png')))

                                door_count += 1 
            
        wall += 1  
    if len(doorNodes) >= 1:
        for n in doorNodes:
            pointList=[]
            points=np.asarray(n.axis.points)
            # pointList.extend(points+n.sign*n.normal*n.width/2)
            pointList.extend(points+n.host.normal*n.host.width/2)

            # pointList.extend(points-n.sign*n.normal*n.width/2)
            pointList.extend(points-n.host.normal*n.host.width/2)

            pointList.extend(np.array(pointList)+np.array([0,0,n.height]))
            pcd=o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(pointList))

            box=pcd.get_oriented_bounding_box()
            n.resource = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(box)
            n.resource.paint_uniform_color(ut.literal_to_array(n.host.color))
            n.doorBox=o3d.geometry.LineSet.create_from_oriented_bounding_box(box)
            n.doorBox.paint_uniform_color([0,0,1])
            
            n.center = t8.compute_center(n.startpoint, n.endpoint)
            n.cartesianTransform = copy.deepcopy(n.host.cartesianTransform)
            n.cartesianTransform[:3,3] = n.center
            n.normal = n.host.normal
            n.rotation = t8.get_angle_with_x_axis(n.startpoint, n.endpoint)

    if len(doorNodes) >= 1:  
        # Function to calculate the distance between two points
        def calculate_distance(point1, point2):
            return np.linalg.norm(np.array(point1) - np.array(point2))


        avg_doorWidth = np.mean(np.array([e.doorWidth for e in doorNodes]))
        #remove single faced doors that are in a 90 degree angle with double faced doors.
        dbscan = DBSCAN(eps=1.5*avg_doorWidth, min_samples=1, metric=t8.calculate_distance) #eps=1.5*avg_doorWidth
        labels = dbscan.fit_predict([n.center for n in doorNodes])

        cluster_ids = [-1] * len(doorNodes)
        unique_labels = set(labels)

        current_cluster_id = 0
        for label in unique_labels:
            cluster_lines = [doorNodes[i] for i in range(len(doorNodes)) if labels[i] == label]
            for line in cluster_lines:
                line_index = None
                for i, l in enumerate(doorNodes):
                    if np.array_equal(l, line):
                        line_index = i
                        break
                cluster_ids[line_index] = current_cluster_id
            current_cluster_id += 1

        all_clusters_indices = []

        # Get unique cluster labels
        unique_labels = np.unique(cluster_ids)

        # Iterate through each unique cluster label
        for cluster_label in unique_labels:
            if not cluster_label == -1:
                cluster_indices = [i for i, label in enumerate(cluster_ids) if label == cluster_label]
                all_clusters_indices.append(cluster_indices)
            else: 
                for i, label in enumerate(cluster_ids):
                    if label == cluster_label:
                        all_clusters_indices.append([i])
        FP = []
        for cluster_indices in all_clusters_indices:
            if len(cluster_indices) > 1:
                doors_to_check = []
                for i in cluster_indices:
                    # doors_to_check.append((doorNodes[i].name, doorNodes[i].doornessScore))
                    if doorNodes[i].blackness  < 0.8:
                        # print(f"door: {doorNodes[i].name} will be removed")
                        FP.append(doorNodes[i].subject)

        doorNodes = [doornode for doornode in doorNodes if doornode.subject not in FP]

        door_count = len(doorNodes)
        # print(f'name: {n.name}, score: {n.doornessScore} ,doorWidth: {n.doorWidth}, height: {n.height}')
    
    for doornode in doorNodes:
        print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, doornode.reference_level, doornode.doorWidth, doornode.height))    
    
    print(f"Number of doors: {len(doorNodes)}")
    
    t8.write_obj_with_submeshes(os.path.join(output_folder,f'{ut.get_filename(f_pcd)}_doors.obj') , [n.resource for n in doorNodes], [n.name for n in doorNodes])
    
    reform_name='_'.join(ut.get_filename(f_pcd).split('_')[:4])+'_doors'

    json_data=t8.doors_to_json(doorNodes)
    with open(os.path.join(output_folder,reform_name+'.json'), 'w') as file:
        json.dump(json_data, file, indent=4)
    print("JSON data written to file:", os.path.join(output_folder,reform_name+'.json'))
 

Doors_0 : Door Score: 0.8269557893276216 => Referencelevel: 0.02; Width: 0.94; height: 2.01
Doors_1 : Door Score: 0.6250131042798361 => Referencelevel: 0.03; Width: 0.98; height: 1.98
Doors_2 : Door Score: 0.8070742080609005 => Referencelevel: 0.02; Width: 0.93; height: 2.01
Doors_3 : Door Score: 0.7551244995660252 => Referencelevel: 0.0; Width: 0.88; height: 1.95
Doors_4 : Door Score: 0.8452296296755474 => Referencelevel: 0.02; Width: 0.9; height: 2.02
Doors_5 : Door Score: 0.8410633444786073 => Referencelevel: 0.02; Width: 0.93; height: 2.01
Doors_6 : Door Score: 0.9774158401069819 => Referencelevel: 0.01; Width: 0.6; height: 2.12
Doors_8 : Door Score: 0.5762718226512274 => Referencelevel: 0.01; Width: 0.98; height: 1.94
Doors_11 : Door Score: 0.990124839047591 => Referencelevel: 0.01; Width: 0.92; height: 2.04
Doors_12 : Door Score: 0.7818506250778835 => Referencelevel: 0.02; Width: 0.94; height: 2.03
Doors_13 : Door Score: 0.6761306897799175 => Referencelevel: 0.0; Width: 0.85; hei

Doors_0 : Door Score: 0.777350722750028 => Referencelevel: 0.0; Width: 0.8; height: 2.25
Doors_1 : Door Score: 0.860120737552643 => Referencelevel: 0.01; Width: 0.91; height: 2.02
Doors_2 : Door Score: 0.8588178197542827 => Referencelevel: 0.01; Width: 0.87; height: 2.01
Doors_3 : Door Score: 0.6845021277666092 => Referencelevel: 0.02; Width: 1.07; height: 2.3
Doors_4 : Door Score: 0.8558429817358654 => Referencelevel: 0.02; Width: 0.92; height: 2.03
Doors_5 : Door Score: 0.6919818814705919 => Referencelevel: 0.0; Width: 0.87; height: 1.89
Doors_6 : Door Score: 0.6883397453063648 => Referencelevel: 0.0; Width: 1.84; height: 2.41
Doors_7 : Door Score: 0.9931517185436356 => Referencelevel: 0.0; Width: 0.96; height: 2.07
Doors_11 : Door Score: 0.9917602406607735 => Referencelevel: 0.01; Width: 0.91; height: 2.0
Doors_12 : Door Score: 0.7875632271501755 => Referencelevel: 0.02; Width: 0.9; height: 1.99
Doors_13 : Door Score: 0.6360830970574716 => Referencelevel: 0.15; Width: 0.73; height: 